In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint
from glob import glob
import pkg_resources

from pymedext_eds.annotators import Endlines, SentenceTokenizer, Hypothesis, \
                                    ATCDFamille, SyntagmeTokenizer, Negation, RegexMatcher, \
                                    QuickUMLSAnnotator, Pipeline
from pymedext_eds.utils import rawtext_loader
from pymedext_eds.viz import display_annotations

In [78]:
from pymedextcore.document import Document

In [90]:
doc = Document("""
Ariel adore le fromage.
Sonia fait de très bons macarons.
""")

In [91]:
doc.annotations

In [92]:
pipeline.annotate([doc])

[{'annotations': [{'type': 'raw_text',
    'value': '\nAriel adore le fromage.\nSonia fait de très bons macarons.\n',
    'ngram': None,
    'span': (0, 59),
    'source': None,
    'source_ID': None,
    'isEntity': False,
    'attributes': None,
    'ID': 'ab99f4be-4575-11eb-864c-3b15d0b9c6c7'},
   {'type': 'endlines',
    'value': '. Ariel adore le fromage.. Sonia fait de très bons macarons.. ',
    'ngram': None,
    'span': (0, 62),
    'source': 'endlines:v0',
    'source_ID': 'ab99f4be-4575-11eb-864c-3b15d0b9c6c7',
    'isEntity': False,
    'attributes': None,
    'ID': 'ac829390-4575-11eb-864c-3b15d0b9c6c7'},
   {'type': 'sentence',
    'value': ' Ariel adore le fromage',
    'ngram': None,
    'span': (1, 24),
    'source': 'sentenceTokenizer:v0',
    'source_ID': 'ac829390-4575-11eb-864c-3b15d0b9c6c7',
    'isEntity': False,
    'attributes': {'hypothesis': 'certain', 'context': 'patient'},
    'ID': 'ac829391-4575-11eb-864c-3b15d0b9c6c7'},
   {'type': 'sentence',
    'value

In [93]:
doc.annotations

In [72]:
data_path = pkg_resources.resource_filename('pymedext_eds', 'data/demo')
file_list = glob(data_path + '/*.txt')
chunk = [rawtext_loader(x) for x in file_list]

In [61]:
import json

In [73]:
doc = chunk[0]

In [74]:
doc.annotations

In [75]:
ann = doc.annotations[0]

In [89]:
endlines = Endlines(['raw_text'], 'endlines', 'endlines:v0')
sentences = SentenceTokenizer(['endlines'], 'sentence', 'sentenceTokenizer:v0')
hypothesis = Hypothesis(['sentence'], 'hypothesis', 'hypothesis:v0')
family = ATCDFamille(['sentence'], 'context', 'ATCDfamily:v0')
syntagmes = SyntagmeTokenizer(['sentence'], 'syntagme', 'SyntagmeTokenizer:v0')
negation = Negation(['syntagme'], 'negation', 'Negation:v0')
regex = RegexMatcher(['raw_text', 'syntagme'], 'regex', 'RegexMatcher:v0', 'patterns.json')

pipeline = Pipeline(pipeline = [endlines, sentences, hypothesis, family, syntagmes, negation, regex])

In [65]:
annotated_chunk = pipeline.annotate(chunk)

In [66]:
pprint(chunk[0].get_annotations('regex')[1].to_dict())

{'ID': 'ab3d5429-4570-11eb-864c-3b15d0b9c6c7',
 'attributes': {'context': 'patient',
                'hypothesis': 'certain',
                'id_regexp': 'arythmie',
                'label': 'arythmie',
                'negation': 'aff',
                'snippet': 'otension artérielle associée à des passages en '
                           'arythmie par fibrillation auriculaire, à une '
                           'anurie et à une insuffisance respira',
                'version': 'v1'},
 'isEntity': True,
 'ngram': None,
 'source': 'RegexMatcher:v0',
 'source_ID': 'ab3d541d-4570-11eb-864c-3b15d0b9c6c7',
 'span': (6019, 6027),
 'type': 'regex',
 'value': 'arythmie'}


In [67]:
record = chunk[0].get_annotations('regex')[1].to_dict()

In [68]:
def to_omop(record):
    s, e = record['span']
    
    attr = record['attributes']
    
    d = dict(
        lexical_variant=record['value'],
        start=s,
        end=e,
        offset=f"{s},{e}",
        snippet=attr['snippet'],
        term_modifiers=','.join([f'{k}={v}' for k, v in attr.items() if k not in {'snippet', 'label'}])
    )
    
    return d

In [69]:
to_omop(record)

{'lexical_variant': 'arythmie',
 'start': 6019,
 'end': 6027,
 'offset': '6019,6027',
 'snippet': 'otension artérielle associée à des passages en arythmie par fibrillation auriculaire, à une anurie et à une insuffisance respira',
 'term_modifiers': 'version=v1,id_regexp=arythmie,hypothesis=certain,context=patient,negation=aff'}

In [70]:
import pandas as pd

In [71]:
pd.DataFrame.from_records([
    to_omop(ann.to_dict())
    for ann in chunk[0].get_annotations('regex')
]).sort_values('start')

,lexical_variant,start,end,offset,snippet,term_modifiers
0,arythmie,3105,3113,"3105,3113","lle à 60/40 mmHg, une fréquence cardiaque à 12...","version=v1,id_regexp=arythmie,hypothesis=certa..."
1,arythmie,6019,6027,"6019,6027",otension artérielle associée à des passages en...,"version=v1,id_regexp=arythmie,hypothesis=certa..."


In [43]:
display_annotations(chunk[0], ['regex'], attributes = ['negation', 'hypothesis', 'context'], label_key = 'label')

In [47]:
chunk[0].annotations[1].to_dict()

{'type': 'endlines',
 'value': 'Le 9 décembre 2010, une femme de 79 ans reçoit un traitement par Pyostacine® pour, semble-t-il, un ulcère variqueux infecté. Cette patiente vit chez son fils. Elle est autonome pour les actes de la vie quotidienne et se déplace avec l\'aide d\'une canne. Elle bénéficie d\'une aide à la toilette.. Le 10 décembre, apparaissent des lésions cutanées.. Le 13 décembre, la patiente est hospitalisée pour une altération de l\'état général dans un contexte de diarrhée.. Lors de l’admission, il est noté une éruption maculo-papuleuse, localisée au niveau du siège, du thorax, de la racine des cuisses et des genoux, avec un signe de Nikolsky positif   ; tendance de la peau à se décoller par simple pression ou frottement . Un prurit est associé aux lésions. Il n\'est pas retrouvé d\'atteinte des muqueuses.. Le bilan biologique montre un syndrome inflammatoire avec une CRP à 198 mg/L.. Le 14 décembre, les lésions se majorent avec des signes de décollement faisant suspec